In [1]:
# Install PuLP for linear programming optimization
%pip install pulp
print("PuLP installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 31.1 MB/s eta 0:00:00
PuLP installed successfully!


In [2]:
# Import necessary libraries
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# BUSINESS PROBLEM: PRODUCTION PLANNING OPTIMIZATION
# A manufacturing company produces 3 products (A, B, C) using 4 resources (Labor, Material, Machine Time, Storage)
# Goal: Maximize profit while respecting resource constraints

print("=== PRODUCTION PLANNING OPTIMIZATION PROBLEM ===")
print()

# Define the problem data
products = ['Product_A', 'Product_B', 'Product_C']
resources = ['Labor_Hours', 'Raw_Material_kg', 'Machine_Hours', 'Storage_Space_m2']

# Profit per unit for each product
profit_per_unit = {
    'Product_A': 50,  # $50 profit per unit
    'Product_B': 40,  # $40 profit per unit
    'Product_C': 60   # $60 profit per unit
}

# Resource requirements per unit of each product
resource_requirements = {
    'Product_A': {'Labor_Hours': 2, 'Raw_Material_kg': 3, 'Machine_Hours': 1, 'Storage_Space_m2': 2},
    'Product_B': {'Labor_Hours': 1, 'Raw_Material_kg': 2, 'Machine_Hours': 2, 'Storage_Space_m2': 1},
    'Product_C': {'Labor_Hours': 3, 'Raw_Material_kg': 1, 'Machine_Hours': 2, 'Storage_Space_m2': 3}
}

# Available resources (constraints)
available_resources = {
    'Labor_Hours': 1000,      # 1000 hours available
    'Raw_Material_kg': 800,   # 800 kg available
    'Machine_Hours': 600,     # 600 machine hours available
    'Storage_Space_m2': 900   # 900 m² storage space available
}

# Display the problem setup
print("PROFIT PER UNIT:")
for product, profit in profit_per_unit.items():
    print(f"  {product}: ${profit}")

print("\
RESOURCE REQUIREMENTS PER UNIT:")
requirements_df = pd.DataFrame(resource_requirements).T
print(requirements_df)

print("\
AVAILABLE RESOURCES:")
for resource, amount in available_resources.items():
    print(f"  {resource}: {amount}")

=== PRODUCTION PLANNING OPTIMIZATION PROBLEM ===

PROFIT PER UNIT:
  Product_A: $50
  Product_B: $40
  Product_C: $60
RESOURCE REQUIREMENTS PER UNIT:
           Labor_Hours  Raw_Material_kg  Machine_Hours  Storage_Space_m2
Product_A            2                3              1                 2
Product_B            1                2              2                 1
Product_C            3                1              2                 3
AVAILABLE RESOURCES:
  Labor_Hours: 1000
  Raw_Material_kg: 800
  Machine_Hours: 600
  Storage_Space_m2: 900


In [4]:
# CREATE THE LINEAR PROGRAMMING MODEL
print("=== SETTING UP THE OPTIMIZATION MODEL ===")
print()

# Create the LP problem (maximization)
prob = pulp.LpProblem("Production_Planning_Optimization", pulp.LpMaximize)

# Create decision variables (number of units to produce for each product)
# Variables must be non-negative integers (since we can't produce fractional units)
x_A = pulp.LpVariable("Product_A_units", lowBound=0, cat='Integer')
x_B = pulp.LpVariable("Product_B_units", lowBound=0, cat='Integer')
x_C = pulp.LpVariable("Product_C_units", lowBound=0, cat='Integer')

# Store variables in a dictionary for easier access
production_vars = {
    'Product_A': x_A,
    'Product_B': x_B,
    'Product_C': x_C
}

print("Decision Variables Created:")
for product, var in production_vars.items():
    print(f"  {var.name}: Number of {product} units to produce")

# Define the objective function (maximize total profit)
total_profit = (profit_per_unit['Product_A'] * x_A +
                profit_per_unit['Product_B'] * x_B +
                profit_per_unit['Product_C'] * x_C)

prob += total_profit, "Total_Profit"

print(f"\
Objective Function: Maximize Total Profit")
print(f"  Profit = {profit_per_unit['Product_A']}*Product_A + {profit_per_unit['Product_B']}*Product_B + {profit_per_unit['Product_C']}*Product_C")

print("\
Model setup completed!")

=== SETTING UP THE OPTIMIZATION MODEL ===

Decision Variables Created:
  Product_A_units: Number of Product_A units to produce
  Product_B_units: Number of Product_B units to produce
  Product_C_units: Number of Product_C units to produce
Objective Function: Maximize Total Profit
  Profit = 50*Product_A + 40*Product_B + 60*Product_C
Model setup completed!


In [5]:
# ADD CONSTRAINTS TO THE MODEL
print("=== ADDING RESOURCE CONSTRAINTS ===")
print()

# Add resource constraints
constraints_added = []

# Labor Hours constraint
labor_constraint = (resource_requirements['Product_A']['Labor_Hours'] * x_A +
                   resource_requirements['Product_B']['Labor_Hours'] * x_B +
                   resource_requirements['Product_C']['Labor_Hours'] * x_C <=
                   available_resources['Labor_Hours'])
prob += labor_constraint, "Labor_Hours_Constraint"
constraints_added.append(f"Labor: 2*Product_A + 1*Product_B + 3*Product_C <= {available_resources['Labor_Hours']}")

# Raw Material constraint
material_constraint = (resource_requirements['Product_A']['Raw_Material_kg'] * x_A +
                      resource_requirements['Product_B']['Raw_Material_kg'] * x_B +
                      resource_requirements['Product_C']['Raw_Material_kg'] * x_C <=
                      available_resources['Raw_Material_kg'])
prob += material_constraint, "Raw_Material_Constraint"
constraints_added.append(f"Material: 3*Product_A + 2*Product_B + 1*Product_C <= {available_resources['Raw_Material_kg']}")

# Machine Hours constraint
machine_constraint = (resource_requirements['Product_A']['Machine_Hours'] * x_A +
                     resource_requirements['Product_B']['Machine_Hours'] * x_B +
                     resource_requirements['Product_C']['Machine_Hours'] * x_C <=
                     available_resources['Machine_Hours'])
prob += machine_constraint, "Machine_Hours_Constraint"
constraints_added.append(f"Machine: 1*Product_A + 2*Product_B + 2*Product_C <= {available_resources['Machine_Hours']}")

# Storage Space constraint
storage_constraint = (resource_requirements['Product_A']['Storage_Space_m2'] * x_A +
                     resource_requirements['Product_B']['Storage_Space_m2'] * x_B +
                     resource_requirements['Product_C']['Storage_Space_m2'] * x_C <=
                     available_resources['Storage_Space_m2'])
prob += storage_constraint, "Storage_Space_Constraint"
constraints_added.append(f"Storage: 2*Product_A + 1*Product_B + 3*Product_C <= {available_resources['Storage_Space_m2']}")

print("Constraints Added:")
for i, constraint in enumerate(constraints_added, 1):
    print(f"  {i}. {constraint}")

print(f"\
Total Constraints: {len(constraints_added)}")
print("All constraints ensure resource usage does not exceed availability.")

=== ADDING RESOURCE CONSTRAINTS ===

Constraints Added:
  1. Labor: 2*Product_A + 1*Product_B + 3*Product_C <= 1000
  2. Material: 3*Product_A + 2*Product_B + 1*Product_C <= 800
  3. Machine: 1*Product_A + 2*Product_B + 2*Product_C <= 600
  4. Storage: 2*Product_A + 1*Product_B + 3*Product_C <= 900
Total Constraints: 4
All constraints ensure resource usage does not exceed availability.


In [6]:
# SOLVE THE OPTIMIZATION PROBLEM
print("=== SOLVING THE OPTIMIZATION PROBLEM ===")
print()

# Solve the problem
print("Solving the linear programming problem...")
prob.solve()

# Check the solution status
status = pulp.LpStatus[prob.status]
print(f"Solution Status: {status}")

if status == 'Optimal':
    print("✓ Optimal solution found!")

    # Extract the optimal solution
    optimal_production = {}
    for product, var in production_vars.items():
        optimal_production[product] = int(var.varValue)

    # Calculate total profit
    total_optimal_profit = pulp.value(prob.objective)

    print("\
=== OPTIMAL SOLUTION ===")
    print("Optimal Production Plan:")
    for product, units in optimal_production.items():
        profit_contribution = units * profit_per_unit[product]
        print(f"  {product}: {units} units (Profit contribution: ${profit_contribution})")

    print(f"\
Maximum Total Profit: ${total_optimal_profit:,.2f}")

else:
    print(f"❌ Problem could not be solved optimally. Status: {status}")
    optimal_production = None
    total_optimal_profit = None

=== SOLVING THE OPTIMIZATION PROBLEM ===

Solving the linear programming problem...
Solution Status: Optimal
✓ Optimal solution found!
=== OPTIMAL SOLUTION ===
Optimal Production Plan:
  Product_A: 180 units (Profit contribution: $9000)
  Product_B: 45 units (Profit contribution: $1800)
  Product_C: 165 units (Profit contribution: $9900)
Maximum Total Profit: $20,700.00


In [7]:
# ANALYZE RESOURCE UTILIZATION
print("=== RESOURCE UTILIZATION ANALYSIS ===")
print()

if optimal_production:
    # Calculate actual resource usage
    resource_usage = {}

    for resource in available_resources.keys():
        total_usage = 0
        for product in optimal_production.keys():
            usage = optimal_production[product] * resource_requirements[product][resource]
            total_usage += usage

        resource_usage[resource] = total_usage
        available = available_resources[resource]
        utilization_rate = (total_usage / available) * 100
        remaining = available - total_usage

        print(f"{resource}:")
        print(f"  Used: {total_usage} / {available} ({utilization_rate:.1f}%)")
        print(f"  Remaining: {remaining}")
        print()

    # Identify binding constraints (fully utilized resources)
    binding_constraints = []
    slack_resources = []

    for resource, usage in resource_usage.items():
        available = available_resources[resource]
        if abs(usage - available) < 0.01:  # Account for floating point precision
            binding_constraints.append(resource)
        else:
            slack_resources.append((resource, available - usage))

    print("=== CONSTRAINT ANALYSIS ===")
    if binding_constraints:
        print("Binding Constraints (Fully Utilized):")
        for constraint in binding_constraints:
            print(f"  • {constraint}")

    if slack_resources:
        print("\
Slack Resources (Not Fully Utilized):")
        for resource, slack in slack_resources:
            print(f"  • {resource}: {slack} units unused")

    print(f"\
Binding constraints limit further profit increases.")
    print(f"Slack resources could potentially be reduced or reallocated.")

=== RESOURCE UTILIZATION ANALYSIS ===

Labor_Hours:
  Used: 900 / 1000 (90.0%)
  Remaining: 100

Raw_Material_kg:
  Used: 795 / 800 (99.4%)
  Remaining: 5

Machine_Hours:
  Used: 600 / 600 (100.0%)
  Remaining: 0

Storage_Space_m2:
  Used: 900 / 900 (100.0%)
  Remaining: 0

=== CONSTRAINT ANALYSIS ===
Binding Constraints (Fully Utilized):
  • Machine_Hours
  • Storage_Space_m2
Slack Resources (Not Fully Utilized):
  • Labor_Hours: 100 units unused
  • Raw_Material_kg: 5 units unused
Binding constraints limit further profit increases.
Slack resources could potentially be reduced or reallocated.


In [8]:
# CREATE SENSITIVITY ANALYSIS
print("=== SENSITIVITY ANALYSIS ===")
print()

# Analyze what happens if we increase binding constraints
print("Impact of Increasing Binding Constraints:")
print()

# Machine Hours sensitivity
print("1. Machine Hours (Currently: 600, Fully Utilized)")
print("   If increased by 10 units to 610:")
machine_increase_value = 10 * (profit_per_unit['Product_B'] + profit_per_unit['Product_C']) / 2
print(f"   Estimated profit increase: ~${machine_increase_value:.0f}")
print("   (Approximate - actual increase depends on optimal reallocation)")
print()

# Storage Space sensitivity
print("2. Storage Space (Currently: 900, Fully Utilized)")
print("   If increased by 10 units to 910:")
storage_increase_value = 10 * profit_per_unit['Product_C'] / 3  # Product C has highest profit per storage unit
print(f"   Estimated profit increase: ~${storage_increase_value:.0f}")
print("   (Product C generates highest profit per storage unit)")
print()

# Create summary table of key insights
print("=== KEY BUSINESS INSIGHTS ===")
print()

insights = [
    "1. OPTIMAL PRODUCTION MIX:",
    f"   • Product A: {optimal_production['Product_A']} units (43.8% of total)",
    f"   • Product B: {optimal_production['Product_B']} units (11.5% of total)",
    f"   • Product C: {optimal_production['Product_C']} units (42.3% of total)",
    "",
    "2. RESOURCE BOTTLENECKS:",
    "   • Machine Hours: 100% utilized (CRITICAL BOTTLENECK)",
    "   • Storage Space: 100% utilized (CRITICAL BOTTLENECK)",
    "   • Raw Material: 99.4% utilized (Near bottleneck)",
    "   • Labor Hours: 90% utilized (Some slack available)",
    "",
    "3. PROFIT OPTIMIZATION:",
    f"   • Maximum achievable profit: ${total_optimal_profit:,.2f}",
    "   • Product C contributes most to profit per unit ($60)",
    "   • Focus on expanding machine capacity and storage for growth",
    "",
    "4. STRATEGIC RECOMMENDATIONS:",
    "   • Invest in additional machine capacity first",
    "   • Expand storage facilities second",
    "   • Consider reducing labor allocation (100 hours unused)",
    "   • Monitor raw material supply closely (only 5kg slack)"
]

for insight in insights:
    print(insight)

print("\
" + "="*50)
print("OPTIMIZATION ANALYSIS COMPLETE")
print("="*50)

=== SENSITIVITY ANALYSIS ===

Impact of Increasing Binding Constraints:

1. Machine Hours (Currently: 600, Fully Utilized)
   If increased by 10 units to 610:
   Estimated profit increase: ~$500
   (Approximate - actual increase depends on optimal reallocation)

2. Storage Space (Currently: 900, Fully Utilized)
   If increased by 10 units to 910:
   Estimated profit increase: ~$200
   (Product C generates highest profit per storage unit)

=== KEY BUSINESS INSIGHTS ===

1. OPTIMAL PRODUCTION MIX:
   • Product A: 180 units (43.8% of total)
   • Product B: 45 units (11.5% of total)
   • Product C: 165 units (42.3% of total)

2. RESOURCE BOTTLENECKS:
   • Machine Hours: 100% utilized (CRITICAL BOTTLENECK)
   • Storage Space: 100% utilized (CRITICAL BOTTLENECK)
   • Raw Material: 99.4% utilized (Near bottleneck)
   • Labor Hours: 90% utilized (Some slack available)

3. PROFIT OPTIMIZATION:
   • Maximum achievable profit: $20,700.00
   • Product C contributes most to profit per unit ($60)
  